<a href="https://colab.research.google.com/github/Yulianagalvis/Se-ales-y-sistemas/blob/main/Parcial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PARCIAL 3 - SEÑALES Y SISTEMAS

## INTEGRANTES: CRISTIAN ARMANDO CHAMORRO MELO - YULIANA ALEXANDRA GALVIS CARDONA - JUAN MANUEL MEJÍA VASCO


## Instalación de paquetes necesarios para ejecutar

In [ ]:
!pip install sympy --upgrade #actualizar sympy para usar módulo de control
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install streamlit -q #instalación de librerías
!pip install soundfile #librerias descarga Youtube y manejo de audios en python
!pip install yt-dlp
!pip install pydub #instalamos pydub para manipular el audio
!pip install scipy
!pip install youtube-dl
!pip install streamlit pandas numpy matplotlib scikit-learn joblib gdown
!pip install pyngrok

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 76, in resolve
    collected = self.factory.collect_root_requirements(root_reqs)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 538, in collect_root_requirements
    reqs = list(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 494, in _make_requirements_from_install_req
    cand = self._mak

In [ ]:
%%writefile Parcial_3.py
import streamlit as st

# Título de la página
st.title('Parcial 3: Señales y Sistemas')

# Subtítulo y detalles del curso
st.subheader('Profesor: Andrés Marino Álvarez Meza, Ph.D.')
st.write('Departamento de Ingeniería Eléctrica, Electrónica, y Computación\nUniversidad Nacional de Colombia - sede Manizales')

# Resumen de los problemas a resolver
st.header('Resumen de los Enunciados')

# Pregunta 1
st.subheader('1. Sistema Masa, Resorte, Amortiguador')
st.markdown("""
- Encontrar la función de transferencia que caracteriza el sistema masa-resorte-amortiguador.
- Modelar un sistema equivalente con un circuito RLC.
- Presentar un dashboard que permita simular el sistema en diferentes condiciones (subamortiguado, amortiguamiento crítico, sobreamortiguado, etc.).
- Incluir gráficas de polos y ceros, diagrama de Bode, respuesta impulso y escalón.
""")

# Pregunta 2
st.subheader('2. Modulación en Amplitud (AM)')
st.markdown("""
- Encontrar el espectro de la señal modulada en amplitud.
- Descargar un fragmento de una canción favorita y simular la modulación AM.
- Presentar gráficas de las señales en tiempo y frecuencia.
- Implementar una simulación para el proceso de demodulación en amplitud.
- Reproducir los fragmentos de audio de mensaje, portadora y señal modulada, e identificar el género musical del audio.
""")


Overwriting Parcial_3.py


## Crear carpeta con los puntos desarrollados

In [ ]:
#Crear directorio pages
import os
try:
    os.mkdir('pages')
except:
    pass

## Sistema masa-resorte / circuito RLC

Encuentre la función de transferencia que caracteriza el sistema presentado en la Figura (asuma condiciones iniciales cero):

![Péndulo Elástico Amortiguado](https://github.com/amalvarezme/SenalesSistemas/blob/master/4_Transformada_Laplace/damped_spring.png?raw=1)

Posteriormente, encuentre el sistema equivalente del modelo masa, resorte, amortguador, a partir del siguiente circuito eléctrico:

![Circuito RLC](https://github.com/amalvarezme/SenalesSistemas/blob/master/Otros/circuitoRLC.jpeg?raw=1)


Luego, encuentre el sistema equivalente a partir de un circuito RLC (entrada de tension - salida tensión del capacitor).
Finalmente, desde el modelado desarrollado, presente un
dashboard que permita simular el sistema como subamortiguado, con amortiguamiento crítico, sobreamortiguado, oscilatorio e inestable. El usuario podra determinar el valor del
factor de amortiguamiento y la simulacion deber  a determi-
nar los valores de m, k y c y sus equivalentes R, L y C;
ademas de estimar el tipo de amortiguamiento. Asimismo,
debera presentar las gráficas de polos y ceros, diagrama de
Bode, respuesta impulso y respuesta al escalon. Nota: Para las graficas de Bode, respuesta impulso y respuesta al es-
calon, se sugiere utilizar la API de control de Sympy, a partir
de los cuadernos guía: Pendulo el  astico y SLIT y fracciones
parciales.


In [ ]:
%%writefile pages/Amortiguador.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy.signal import TransferFunction as tf, bode, impulse, step
from sympy.physics.control.lti import TransferFunction
from sympy.physics.control.control_plots import pole_zero_plot
import warnings

# Ignorar advertencias
warnings.filterwarnings("ignore")

# Definición de variables simbólicas
t, m, c, k, s = sym.symbols('t m c k s', real=True, positive=True)

# Título y descripción principal
st.title("Simulación de Sistema Masa-Resorte-Amortiguador y Clasificación del Tipo de Amortiguamiento")
st.write("""
Este simulador te permite visualizar cómo funciona un sistema masa-resorte-amortiguador
y clasificar el tipo de amortiguamiento en función de los parámetros seleccionados.
Puedes modificar los valores de la masa, la constante del resorte y el factor de amortiguamiento (ξ).
""")

# Explicación de la fórmula para ξ
st.write("""
El factor de amortiguamiento (ξ) es un parámetro que determina el tipo de amortiguamiento en el sistema.
La fórmula que relaciona ξ con la masa (m), la constante del resorte (k), y el coeficiente de amortiguamiento (c) es la siguiente:
""")
st.latex(r'\xi = \frac{c}{2\sqrt{k m}}')
# Explicación del cálculo de c
st.write("""
El coeficiente de amortiguamiento (c) se calcula usando los valores seleccionados para m, k, y ξ.
Este valor influye directamente en el comportamiento del sistema.
""")

# Determinación del tipo de amortiguamiento
st.write("""
Dependiendo del valor del factor de amortiguamiento (ξ), el sistema puede estar en uno de los siguientes estados:
- **Subamortiguado**: ξ < 1
- **Amortiguamiento crítico**: ξ = 1
- **Sobreamortiguado**: ξ > 1
""")

# Definir la función de transferencia simbólica
st.subheader("Función de Transferencia")
st.write("""
La función de transferencia representa la relación entre la salida y la entrada del sistema en el dominio de la frecuencia.
Es importante para analizar la estabilidad y el comportamiento dinámico del sistema.
""")

# Entrada del usuario: Factor de amortiguamiento
xi_val = st.slider("Seleccione el valor del Factor de Amortiguamiento (ξ)", 0.0, 2.0, 0.5)

# Definir un rango para `k` y `m` para explorar diferentes combinaciones
k_min, k_max = 1.0, 10.0
m_min, m_max = 0.5, 5.0
# Mostrar sliders para seleccionar los valores de `k` y `m`
k_val = st.slider("Seleccione un valor para la constante del resorte (k)", k_min, k_max, 5.0)
m_val = st.slider("Seleccione un valor para la masa (m)", m_min, m_max, 1.0)

c_val = 2 * xi_val * np.sqrt(k_val * m_val)

# Mostrar los valores calculados
st.subheader(f"**Coeficiente de Amortiguamiento (c)**: {c_val:.2f}")

# Mostrar intervalo permitido para cada parámetro que mantenga el tipo de amortiguamiento
st.subheader("Intervalos permitidos para el mismo tipo de sistema:")
st.write("""
Aquí se muestran los valores mínimos y máximos del coeficiente de amortiguamiento (c) que mantienen
el tipo de amortiguamiento seleccionado.
""")

# Calcular los valores mínimos y máximos de c para cada combinación
c_min = 2 * 0.01 * np.sqrt(k_val * m_val)  # Mínimo coeficiente de amortiguamiento para ξ = 0.01
c_max = 2 * 2.0 * np.sqrt(k_val * m_val)  # Máximo coeficiente de amortiguamiento para ξ = 2.0

st.write(f"**Rango de `c` que preserva el tipo de amortiguamiento**: {c_min:.2f} ≤ c ≤ {c_max:.2f}")

if xi_val < 1:
    st.write("El sistema está **subamortiguado** (ξ < 1).")
elif xi_val == 1:
    st.write("El sistema tiene **amortiguamiento crítico** (ξ = 1).")
else:
    st.write("El sistema está **sobreamortiguado** (ξ > 1).")

H = (1/k_val) * (s**2) / (s**2 + 2 * xi_val * np.sqrt(k_val / m_val) * s + (k_val / m_val))

# Crear la función de transferencia usando scipy para simulaciones
system = tf([1], [1, 2 * xi_val * np.sqrt(k_val / m_val), k_val / m_val])

# Gráfica de polos y ceros con sympy
st.subheader("Gráfica de Polos y Ceros")
st.write("""
La gráfica de polos y ceros es fundamental para determinar la estabilidad del sistema.
Los polos son los valores donde la salida del sistema tiende al infinito, y los ceros son donde la salida es cero.
""")
fig1 = plt.figure()
tf1 = TransferFunction(*sym.fraction(H), s)
pole_zero_plot(tf1)
st.pyplot(fig1)

# Diagrama de Bode con scipy
st.subheader("Diagrama de Bode")
st.write("""
El diagrama de Bode muestra la respuesta del sistema en términos de magnitud y fase en función de la frecuencia.
Es útil para entender cómo el sistema responde a diferentes frecuencias.
""")
fig2, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
w, mag, phase = bode(system)
ax1.semilogx(w, mag)  # Magnitud
ax1.set_title('Bode Plot')
ax1.set_ylabel('Magnitud (dB)')
ax2.semilogx(w, phase)  # Fase
ax2.set_ylabel('Fase (grados)')
ax2.set_xlabel('Frecuencia (rad/s)')
st.pyplot(fig2)

# Respuesta al impulso con scipy
st.subheader("Respuesta al Impulso")
st.write("""
La respuesta al impulso muestra cómo el sistema responde a una entrada tipo impulso.
Es una medida de la estabilidad y rapidez de la respuesta del sistema.
""")
t, y_impulse = impulse(system)
fig3 = plt.figure()
plt.plot(t, y_impulse)
plt.title("Respuesta al Impulso")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
st.pyplot(fig3)

# Respuesta al escalón con scipy
st.subheader("Respuesta al Escalón")
st.write("""
La respuesta al escalón muestra cómo el sistema responde a una entrada tipo escalón.
Es útil para analizar la estabilidad, el sobreimpulso, y el tiempo de asentamiento del sistema.
""")
t, y_step = step(system)
fig4 = plt.figure()
plt.plot(t, y_step)
plt.title("Respuesta al Escalón")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
st.pyplot(fig4)

# Conclusión general
st.subheader("Conclusión General")
st.write("""
Este simulador de **sistema masa-resorte-amortiguador** proporciona una manera interactiva de visualizar y comprender el comportamiento dinámico de un sistema físico sujeto a fuerzas de amortiguamiento y elasticidad. A través de la variación de parámetros como la masa, la constante del resorte, y el factor de amortiguamiento (ξ), se puede explorar cómo el sistema responde bajo diferentes condiciones, clasificando el tipo de amortiguamiento en tres categorías principales:

1. **Subamortiguado (ξ < 1)**: El sistema oscila antes de estabilizarse, con un tiempo de asentamiento más largo y presencia de sobreimpulso.
2. **Amortiguamiento crítico (ξ = 1)**: El sistema alcanza la estabilidad sin oscilaciones y en el menor tiempo posible.
3. **Sobreamortiguado (ξ > 1)**: El sistema se estabiliza lentamente, sin oscilaciones, pero con un tiempo de respuesta más largo.

Además, las herramientas gráficas como los **diagramas de polos y ceros**, **diagramas de Bode**, y las respuestas al **impulso** y **escalón**, permiten un análisis visual del comportamiento del sistema frente a diferentes entradas, proporcionando una mejor comprensión de su estabilidad, tiempo de respuesta y comportamiento en el dominio de la frecuencia.

Este enfoque ofrece una manera eficaz de simular el comportamiento de sistemas físicos reales y sugiere aplicaciones útiles en el diseño de sistemas de control en ingeniería, donde la elección adecuada del amortiguamiento es crucial para el rendimiento óptimo.
""")
st.write("Además recordamos que porlas equivalencias con el sistema RLC-serie:")
st.write(f"R = {c_val}")
st.write(f"L = {m_val}")
st.write(f"C = {1/k_val}")


Overwriting pages/Amortiguador.py


## Modulación - Demodulación

Sea la señal portadora c(t) = Ac sin(2πFct), con Ac, Fc ∈ R, y la señal mensaje m(t) ∈ R. Encuentre el espectro en frecuencia de la señal modulada en amplitud (AM), y(t) = (1 + m(t)/Ac) c(t).
Luego, descargue desde YouTube 5 segundos de su canción favorita (capturando del segundo 20 al 25). Presente una simulación de modulación por amplitud AM (tomando como mensaje el fragmento de la canción escogida). Grafique las señales en tiempo y frecuencia (magnitud y fase) de la señal mensaje, portadora y modulada. Reproduzca los fragmentos de audio del mensaje, portadora y señal modulada.
Nota: se sugiere utilizar un canal de señal de audio para el desarrollo del ejercicio.
El usuario debe poder escoger el índice de modulación deseado.


Sea el demodulador en amplitud presentado en la siguiente figura:

![Circuito RLC](https://github.com/amalvarezme/SenalesSistemas/blob/master/Otros/AM.jpeg?raw=1)

Asumiendo $\theta_0 = 0$, presente una simulacion sobre Python
para el proceso de demodulacion en amplitud. Especifique
adecuadamente las consideraciones de diseño en tiempo y en
frecuencia (con las graficas pertinentes), y reproduzca el seg-
mento de la cancion para cada una de las etapas del sistema
de modulacion y demodulación. Tenga en cuenta el diseño
de un filtro digital pasa bajas en la etapa de demodulacion,
utilizando Transformada Z. Presente la grafica de polos y
ceros y el diagrama de Bode del filtro (se sugiere tener en
cuenta el cuaderno Transformada Z).
Finalmente, la simulacion tipo dashboard debería también
permitir la reproduccion de los fragmentos de audio del mensaje, portadora y senal modulada, y detectar el tipo de género musical del audio empleado. Para ello se deben escoger
al menos dos generos musicales en la etapa de entrenamiento del sistema. Nota: se sugiere utilizar un canal de senal de
audio para el desarrollo del ejercicio. El usuario debe poder
escoger el índice de modulacion deseado

In [ ]:
%%writefile pages/Modulacion.py
import streamlit as st
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import io
import joblib
import matplotlib.pyplot as plt
import subprocess
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.signal import butter, filtfilt, freqz_zpk
from pydub import AudioSegment
from yt_dlp import YoutubeDL
from matplotlib.patches import Circle

# Función para descargar audio de YouTube y convertirlo a WAV
def descargar_audio_youtube(url):
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]',
        'outtmpl': 'audio.m4a'
    }
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    audio = AudioSegment.from_file("audio.m4a", format="m4a")
    audio.export("audio.wav", format="wav")

# Función para graficar la respuesta de frecuencia de un filtro digital
def plot_freq_response(filter_name, w, h, N):
    fig, ax1 = plt.subplots()
    ax1.set_title(filter_name + '.' + str(N))
    ax1.plot(w, 20 * np.log10(abs(h)), 'b')
    ax1.set_ylabel('Amplitud [dB]', color='b')
    ax1.set_xlabel('Frecuencia[Hz]')
    ax1.grid()
    ax2 = ax1.twinx()
    angles = np.unwrap(np.angle(h))
    ax2.plot(w, angles, 'g')
    ax2.set_ylabel('Ángulo [radianes]', color='g')
    st.pyplot(fig)

# Función para mostrar polos y ceros en el plano Z
def show_zp(z, p, title='Plano Z'):
    fig, ax = plt.subplots()
    ax.plot(np.real(z), np.imag(z), 'bo', fillstyle='none', ms=10)
    ax.plot(np.real(p), np.imag(p), 'rx', fillstyle='none', ms=10)
    unit_circle = Circle((0, 0), radius=1, fill=False, color='black', ls='solid', alpha=0.9)
    ax.add_patch(unit_circle)
    ax.axvline(0, color='0.7')
    ax.axhline(0, color='0.7')
    ax.set_title(title)
    ax.set_xlabel(r'Re{$z$}')
    ax.set_ylabel(r'Im{$z$}')
    ax.axis('equal')
    ax.set_xlim((-2, 2))
    ax.set_ylim((-2, 2))
    ax.grid()
    st.pyplot(fig)

# Título de la aplicación
st.title('Simulación de Modulación AM y Demodulación')

# Input para el enlace de YouTube
url = st.text_input('Ingrese el enlace de YouTube de la canción que desea procesar')

# Descargar y convertir el audio si se proporciona un enlace
try:
  os.remove("audio.m4a")
  os.remove("audio.wav")
except:
  pass

if url:
    with st.spinner('Descargando y procesando el audio...'):
        descargar_audio_youtube(url)
        st.success('Descarga y conversión completa.')

    st.markdown("### Procesamiento de la Señal Original")
    st.write("En este paso se descarga el audio desde YouTube y se convierte en un archivo WAV para su procesamiento posterior.")

    # Cargar archivo WAV

    audio_file = "audio.wav"
    xa, fs = sf.read(audio_file)
    st.write(f"Frecuencia de muestreo: {fs} Hz")

    # Extraer segmento de audio
    ti = 20  # Tiempo inicial en segundos
    tf = 25  # Tiempo final en segundos
    m = xa[int(ti*fs):int(tf*fs), 0]
    tt = np.arange(0, len(m)) / fs

    st.markdown("### Visualización de la Señal en el Tiempo")
    st.write("Graficamos la señal de audio en función del tiempo para observar su forma de onda y analizarla visualmente.")
    fig, ax = plt.subplots()
    ax.plot(tt, m)
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('Amplitud')
    st.pyplot(fig)

    # Reproducir señal original
    wav_buffer = io.BytesIO()
    sf.write(wav_buffer, m, fs, format='wav')
    wav_buffer.seek(0)
    st.audio(wav_buffer, format='audio/wav')

    st.markdown("### Espectro de la Señal Original")
    st.write("A través de la transformada de Fourier, convertimos la señal original al dominio de la frecuencia para ver su distribución espectral.")
    Mw = np.fft.rfft(m)
    vf = np.fft.rfftfreq(m.shape[0], 1/fs)

    fig, ax = plt.subplots()
    ax.plot(vf, np.abs(Mw))
    ax.set_title('Magnitud del espectro')
    ax.set_xlabel('Frecuencia [Hz]')
    ax.set_ylabel('|M(f)|')
    st.pyplot(fig)

    st.markdown("### Generación de la Portadora")
    st.write("Definimos una frecuencia para la portadora, la cual es una señal sinusoidal que usaremos para modular la señal original.")
    Fo = st.number_input('Frecuencia de la portadora (Hz)', min_value=1000, max_value=50000, value=15000)
    c = np.cos(2 * np.pi * Fo * tt)

     # Reproducir señal portadora
    wav_buffer = io.BytesIO()
    sf.write(wav_buffer, c, fs, format='wav')
    wav_buffer.seek(0)
    st.audio(wav_buffer, format='audio/wav')

    st.markdown("### Modulación AM")
    st.write("Aplicamos modulación en amplitud (AM) multiplicando la señal original por la portadora.")
    # Selección del índice de modulación
    mu = st.slider('Índice de modulación (μ)', min_value=0.0, max_value=2.0, value=1.0, step=0.1)
    A1 = 2  # Amplitud de la portadora
    y = (1 + mu * m) * c

    # Graficar señal modulada
    st.subheader('Señal modulada AM en el tiempo')
    fig, ax = plt.subplots()
    ax.plot(tt, y)
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('Amplitud')
    st.pyplot(fig)

    #Reproducir Señal modulada
    wav_buffer = io.BytesIO()
    sf.write(wav_buffer, y, fs, format='wav')
    wav_buffer.seek(0)
    st.audio(wav_buffer, format='audio/wav')

    st.markdown("### Demodulación")
    st.write("Multiplicamos la señal modulada por la misma portadora para recuperar la señal original, que queda 'envuelta' en la portadora.")
    d = y * c  # Señal demodulada

    # Reproducir señal demodulada
    wav_buffer = io.BytesIO()
    sf.write(wav_buffer, d, fs, format='wav')
    wav_buffer.seek(0)
    st.audio(wav_buffer, format='audio/wav')

    # Graficar señal demodulada
    st.subheader('Señal demodulada en el tiempo')
    fig, ax = plt.subplots()
    ax.plot(tt, d)
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('Amplitud')
    st.pyplot(fig)

    st.markdown("### Filtrado de la Señal Demodulada")
    st.write("Se utiliza un filtro pasa-bajas para eliminar la portadora y recuperar la señal de mensaje original.")
    # Filtro pasa-bajas
    N = 10
    Wn = 14950  # Frecuencia de corte
    zeros, poles, gain = butter(N, Wn, btype='low', output='zpk', fs=fs)
    w, h = freqz_zpk(zeros, poles, gain, fs=fs)

    # Graficar respuesta del filtro
    plot_freq_response('Filtro Butterworth', w, h, N)

    # Graficar polos y ceros
    show_zp(zeros, poles)

    # Filtrar señal demodulada
    num, den = butter(N, Wn, btype='low', fs=fs)
    xf = filtfilt(num, den, d)
    me = (2 / A1) * xf  # Señal recuperada

    st.markdown("### Resultados Finales")
    st.write("Finalmente, se reproduce y se grafica la señal recuperada.")
    wav_buffer = io.BytesIO()
    sf.write(wav_buffer, me, fs, format='wav')
    wav_buffer.seek(0)
    st.audio(wav_buffer, format='audio/wav')
    fig, ax = plt.subplots()
    ax.plot(tt, me)
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('Amplitud')
    st.pyplot(fig)



Overwriting pages/Modulacion.py


## Detector

In [ ]:
%%writefile pages/Detector.py
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np
import joblib
import os
import subprocess
import soundfile as sf
import yt_dlp as youtube_dl
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import math
import io

# Configuración inicial
st.title("Clasificador de Canciones: Bachata vs Metal")
# Explicación del modelo
st.markdown("""
### Teoría de Clasificación de Géneros Musicales con Machine Learning
En el campo de la **clasificación de audio**, el objetivo es identificar automáticamente el género de una canción basándose en características extraídas de la señal. Para esto, se construyen modelos de machine learning que analizan patrones en el dominio de las **frecuencias** y el **tiempo** para diferenciar entre distintas categorías musicales.
""")
model_path = "bachata_vs_metal.pkl"

# Cargar el modelo cada vez que se proporciona un nuevo link
@st.cache_data
def cargar_modelo():
    return joblib.load(model_path)

# Función para descargar el archivo MP3 desde YouTube
def download_ytvid_as_mp3(video_url, name):
  """
  Utiliza la librería `yt_dlp` para descargar solo el audio de un video de YouTube.
  Esto se hace porque los modelos de análisis de audio requieren el formato WAV para una mayor precisión.
  """
  video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
  filename = f"{name}.mp3"
  options = {
      'format': 'bestaudio/best',
      'keepvideo': False,
      'outtmpl': filename,
  }
  with youtube_dl.YoutubeDL(options) as ydl:
      ydl.download([video_info['webpage_url']])

  return filename

# Input del usuario
video_url = st.text_input("Ingrese el link de YouTube para la canción:")
if video_url:
  # Inicializar variables
  my_model_loaded = cargar_modelo()  # Cargar el modelo desde el archivo cada vez
  name_ = 'results/Nueva_cancion'

  # Descargar y convertir la canción
  if not os.path.exists('results'):
      os.mkdir('results')
  try:
    os.remove('results/Nueva_cancion.mp3')

    os.remove('results/Nueva_cancion.wav')
  except FileNotFoundError:
    pass
  download_ytvid_as_mp3(video_url, name_)
  subprocess.call(['ffmpeg', '-y', '-i', name_+'.mp3', name_+'.wav'])

  # Leer el archivo WAV
  st.markdown("""### Extracción de características de audio
  La señal de audio es una función del tiempo que contiene información acerca de la **amplitud** y **frecuencia** de la onda sonora. Para analizar la estructura de la música, primero segmentamos el archivo en fragmentos más pequeños. De esta manera, podemos capturar cambios locales en el espectro que son indicativos del estilo musical.
  """)
  fs = 48000
  tl = np.array([40, 50, 60, 70, 80, 90, 100])  # puntos de lectura
  ts = 5  # tiempo segmento
  Ns = 7  # cantidad segmentos
  x_t = np.zeros((Ns, int(ts*fs), 2))  # Ns segmentos, cantidad de muestras, 2 canales (stereo)

  x, fs = sf.read(name_+'.wav')
  for i, ti in enumerate(tl):  # segmentar la canción
      x_t[i] = x[int(fs*ti):int(fs*(ti+ts)), :]

  # Reproducir segmento de la canción
  wav_buffer = io.BytesIO()
  sf.write(wav_buffer, x_t[1], fs, format='wav')
  wav_buffer.seek(0)
  st.audio(wav_buffer, format='audio/wav')


  # Aplicación de la Transformada de Fourier
  st.markdown("""
  ### Análisis de la Transformada de Fourier
  La **Transformada de Fourier** permite descomponer la señal de audio en sus componentes de frecuencia. Esto es crucial en la clasificación de géneros, ya que ciertos estilos musicales (como el Metal) contienen patrones específicos en las bandas de frecuencia altas, mientras que la Bachata tiende a mostrar patrones en bandas de frecuencia más bajas.

  Mediante la **Transformada Rápida de Fourier (FFT)**, calculamos la representación espectral de cada segmento. La frecuencia se representa en el eje horizontal y la magnitud (potencia) en el eje vertical.
  """)
  vf = np.fft.rfftfreq(x_t.shape[1], 1/fs)  # vector de frecuencias
  Xw = np.fft.rfft(x_t, axis=1).mean(axis=-1)  # transformada rápida de Fourier

  # Graficar señales de tiempo y frecuencia
  st.markdown("""
  ### Señal de audio en el tiempo
  """
  )
  fig, ax = plt.subplots()
  ax.plot(np.arange(0, ts, 1/fs), x_t.mean(axis=-1).T)
  ax.set_xlabel('t [s]')
  ax.set_ylabel('x(t)')
  st.pyplot(fig)

  # Graficar la transformada de Fourier
  st.markdown("""
  ### Transformada de Fourier
  """
  )
  fig, ax = plt.subplots()
  ax.plot(vf, abs(Xw).T)
  ax.set_xlabel('f [Hz]')
  ax.set_ylabel('|X(f)|')
  st.pyplot(fig)

  # Normalización de las características espectrales
  st.markdown("""
  ### Normalización de características
  Para que los modelos de machine learning puedan comparar diferentes muestras de audio, es importante que todas las características tengan el mismo rango. Aquí utilizamos la técnica de **normalización Min-Max**, que ajusta los valores entre 0 y 1.
  """)
  sca = MinMaxScaler()
  Xw_ = sca.fit_transform(abs(Xw).T).T
  fig, ax = plt.subplots()
  ax.plot(vf, Xw_.T)
  ax.set_xlabel('f [Hz]')
  ax.set_ylabel('|X(f)| (Normalizado)')
  st.pyplot(fig)

  # Agregar fragmentos al modelo
  my_model_loaded['Xw_'] = np.concatenate([my_model_loaded['Xw_'], Xw_], axis=0)
  label = np.ones((7, 1)) * 1.5
  my_model_loaded['label'] = np.concatenate([my_model_loaded['label'], label], axis=0)
  name_c = ["Nueva_cancion_0" for i in range(7)]
  my_model_loaded['name_c'].extend(name_c)

  # Reducción de dimensionalidad con TSNE
  st.markdown("""
  ### Reducción de dimensionalidad con TSNE
  Los datos de audio tienen un alto número de dimensiones debido a la cantidad de frecuencias analizadas. Para visualizar las relaciones entre canciones, utilizamos **TSNE (T-Distributed Stochastic Neighbor Embedding)**, una técnica de reducción de dimensionalidad que proyecta los puntos en un espacio 2D preservando relaciones de proximidad.
  """)
  red_ = TSNE(perplexity=30, n_components=2, random_state=0, learning_rate='auto', init='pca')
  X_2D = red_.fit_transform(my_model_loaded['Xw_'][:, :my_model_loaded['fmax']])

  # Ubicar puntos cercanos
  vpos = []
  for i in X_2D[-7:]:
    menor = 10000000
    pos = 0
    for countj, j in enumerate(X_2D[:-7]):
      dist = math.dist(i, j)
      if dist < menor:
        menor = dist
        pos = countj
    vpos.append(pos)

  # Graficar puntos 2D
  fig, ax = plt.subplots()
  scatter = ax.scatter(X_2D[:, 0], X_2D[:, 1], c=my_model_loaded['label'], s=10)
  for i in vpos:
      ax.scatter(X_2D[i, 0], X_2D[i, 1], c='r', marker='x')
  fig.colorbar(scatter)
  st.pyplot(fig)

  # Clasificación según puntos cercanos
  st.markdown("""
  ### Clasificación de la canción
  La clasificación se realiza determinando las etiquetas de los puntos más cercanos a los nuevos segmentos. Si la mayoría de los puntos cercanos pertenecen a "Metal", la canción es clasificada como Metal; de lo contrario, es clasificada como Bachata.
  """)
  nmetal = sum(my_model_loaded['label'][i] == 1.0 for i in vpos)
  nbachata = len(vpos) - nmetal

  if nmetal > nbachata:
      st.write("La canción seleccionada es Metal 🎸")
  else:
      st.write("La canción seleccionada es Bachata 💃")

Overwriting pages/Detector.py


## Entrenador

In [ ]:
"""%%writefile pages/Entrenador.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import yt_dlp as youtube_dl
import soundfile as sf
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import joblib
import gdown
from datetime import datetime
import shutil

# Configuración de la página
st.set_page_config(
    page_title="Análisis de Género Musical",
    layout="wide",
    initial_sidebar_state="expanded",
)

st.title("Análisis de Género Musical con Streamlit")

# 1. Descarga de Datos desde Google Drive
st.header("1. Descarga de Datos desde Google Drive")

FILEID = "1DudgUNAZPWv_42o9iOEAKurQQlnuCPNL"
DOWNLOAD_URL = f"https://drive.google.com/uc?id={FILEID}&export=download"

output = "canciones.xlsx"

if not os.path.exists(output):
    with st.spinner("Descargando archivo de Google Drive..."):
        try:
            gdown.download(DOWNLOAD_URL, output, quiet=False)
            st.success("Archivo descargado exitosamente.")
        except Exception as e:
            st.error(f"Error al descargar el archivo: {e}")
else:
    st.info("El archivo 'canciones.xlsx' ya existe.")

# 2. Visualización de los Datos
st.header("2. Visualización de los Datos")

try:
    X = pd.read_excel(output)
    st.write("Vista previa de los datos:")
    st.dataframe(X.head())
except Exception as e:
    st.error(f"Error al leer el archivo Excel: {e}")

# 3. Descarga y Procesamiento de Audios de YouTube
st.header("3. Descarga y Procesamiento de Audios de YouTube")

# Instalación de yt-dlp y soundfile (Si no están instalados)
# Nota: Streamlit puede reiniciar la aplicación después de una instalación, lo que puede interrumpir el flujo.
# Es recomendable instalar las dependencias antes de ejecutar la aplicación.

# Definir funciones necesarias
@st.cache_data
def download_ytvid_as_mp3(video_url, name):
    video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    return filename

# Crear carpeta para resultados
results_dir = 'results'
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    st.info(f"Carpeta '{results_dir}' creada.")
else:
    st.info(f"Carpeta '{results_dir}' ya existe.")

# Botón para iniciar el proceso de descarga y procesamiento
if st.button("Iniciar Descarga y Procesamiento de Audios"):
    N, P = X.shape
    Ns = N * 5  # Cantidad de segmentos por canción

    progress_bar = st.progress(0)
    status_text = st.empty()

    for n in range(N):
        status_text.text(f"Procesando video {n+1} de {N}")
        st.write(f"**Link:** {X.loc[n, 'link']}")
        st.write(f"**Artista:** {X.loc[n, 'artista']}")
        st.write(f"**Género:** {X.loc[n, 'genero']}")

        # Ruta video n-ésimo
        name_ = f"{results_dir}/{X.loc[n, 'artista']}_{n}_{X.loc[n,'tipo']}"

        try:
            # Descargar mp3 desde YouTube
            download_ytvid_as_mp3(X.loc[n, 'link'], name_)
            st.success(f"Descargado: {name_}.mp3")
        except Exception as e:
            st.error(f"Error al descargar el video {n+1}: {e}")
            continue

        try:
            # Convertir a .wav usando ffmpeg
            subprocess.call(['ffmpeg', '-y', '-i', f"{name_}.mp3", f"{name_}.wav"], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
            st.success(f"Convertido a WAV: {name_}.wav")
        except Exception as e:
            st.error(f"Error al convertir a WAV para el video {n+1}: {e}")
            continue

        progress_bar.progress((n + 1) / N)

    st.success("Descarga y conversión de audios completadas.")

# 4. Procesamiento de Audio y Extracción de Características
st.header("4. Procesamiento de Audio y Extracción de Características")

if st.button("Procesar Audios y Extraer Características"):
    path = 'results/'
    wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]
    st.write(f"Archivos WAV encontrados: {len(wav_files)}")

    if len(wav_files) == 0:
        st.warning("No se encontraron archivos WAV para procesar.")
    else:
        # Parámetros de procesamiento
        fs = 48000
        tl = np.array([40,50,60,70,80,90,100])  # Puntos de lectura
        ts = 5  # Duración del segmento en segundos
        Ns = len(wav_files) * len(tl)  # Cantidad de segmentos
        x_t = np.zeros((Ns, int(ts * fs), 2))  # Stereo
        label = np.zeros((Ns, 1))  # Género
        name_c = []

        i = 0
        progress_bar = st.progress(0)
        status_text = st.empty()

        for name in wav_files:
            status_text.text(f"Procesando archivo: {name}")
            try:
                x, fs = sf.read(os.path.join(path, name))
                audio_duration = len(x) / fs

                for ti in tl:
                    if ti + ts <= audio_duration:
                        x_t[i] = x[int(fs * ti):int(fs * (ti + ts)), :2]  # Asegurarse de tener 2 canales
                        label[i] = int(name[-5])  # Suponiendo que el tipo de género está en el nombre del archivo
                        name_c.append(name[:-6])
                        i += 1
                    else:
                        st.warning(f"Omitiendo segmento para {name} en {ti} segundos.")
            except Exception as e:
                st.error(f"Error al procesar {name}: {e}")
                continue

            progress_bar.progress((i) / Ns)

        x_t = x_t[:i]
        label = label[:i]
        name_c = name_c[:i]

        st.success("Procesamiento de audios completado.")
        st.write(f"Forma de x_t: {x_t.shape}")
        st.write(f"Forma de label: {label.shape}")

        # Guardar variables para uso posterior
        model_data = {
            'x_t': x_t,
            'fs': fs,
            'label': label,
            'name_c': name_c
        }

        st.session_state['model_data'] = model_data

# 5. Visualización de Resultados
st.header("5. Visualización de Resultados")

if 'model_data' in st.session_state:
    model_data = st.session_state['model_data']
    x_t = model_data['x_t']
    fs = model_data['fs']
    label = model_data['label']
    name_c = model_data['name_c']

    st.subheader("Reproducir un Segmento de Audio")
    i = st.slider("Selecciona el segmento a reproducir", 0, len(x_t)-1, 0)
    st.audio(x_t[i].T, format='audio/wav', sample_rate=fs)

    st.subheader("Cálculo de la Transformada de Fourier")
    vf = np.fft.rfftfreq(x_t.shape[1], 1/fs)
    Xw = np.fft.rfft(x_t, axis=1).mean(axis=-1)

    st.write(f"Forma de Xw: {Xw.shape}")

    st.subheader("Gráficas de Tiempo y Fourier")
    fig1, ax1 = plt.subplots()
    ax1.plot(np.arange(0, ts, 1/fs), x_t[i].mean(axis=-1).T)
    ax1.set_xlabel('t [s]')
    ax1.set_ylabel('x(t)')
    st.pyplot(fig1)

    fig2, ax2 = plt.subplots()
    ax2.plot(vf, np.abs(Xw[i]).T)
    ax2.set_xlabel('f [Hz]')
    ax2.set_ylabel('|X(f)|')
    st.pyplot(fig2)

    # Normalización de espectros
    sca = MinMaxScaler()
    Xw_ = sca.fit_transform(np.abs(Xw).T).T

    fig3, ax3 = plt.subplots()
    ax3.plot(vf, Xw_.T)
    ax3.set_xlabel('f [Hz]')
    ax3.set_ylabel('|X(f)| (Normalizado)')
    st.pyplot(fig3)

    # En dB
    fig4, ax4 = plt.subplots()
    ax4.plot(vf, (20 * np.log10(Xw_ + 1e-10)).T)
    ax4.set_xlabel('f [Hz]')
    ax4.set_ylabel('|X(f)| dB')
    st.pyplot(fig4)

    # Reducción de Dimensionalidad
    st.subheader("Reducción de Dimensionalidad con t-SNE")

    fmax = 7000
    X_reduced = TSNE(n_components=2, random_state=123, perplexity=20, learning_rate='auto').fit_transform(Xw_[:, :fmax])

    fig5, ax5 = plt.subplots()
    scatter = ax5.scatter(X_reduced[:, 0], X_reduced[:, 1], c=label.squeeze(), cmap='viridis', s=10)
    plt.colorbar(scatter, ax=ax5, label='Género')
    ax5.set_title("Visualización 2D de los Segmentos de Audio")
    st.pyplot(fig5)

    st.success("Visualización completada.")
else:
    st.warning("Por favor, procesa los audios antes de visualizar los resultados.")

# 6. Guardar y Descargar el Modelo
st.header("6. Guardar y Descargar el Modelo")

if 'model_data' in st.session_state and st.button("Guardar Modelo"):
    model_data = st.session_state['model_data']
    x_t = model_data['x_t']
    fs = model_data['fs']
    label = model_data['label']
    name_c = model_data['name_c']

    try:
        # Crear carpeta para el modelo
        modelo_dir = 'modelo'
        if not os.path.exists(modelo_dir):
            os.mkdir(modelo_dir)

        # Guardar el modelo
        filename_ = os.path.join(modelo_dir, 'Bachata_vs_metal.pkl')
        model_ = {
            'Xw_': x_t,  # Deberías ajustar esto según lo que realmente quieres guardar
            'fmax': fmax,
            'label': label,
            'name_c': name_c,
            'vf': vf,
            'fs': fs
        }
        joblib.dump(model_, filename_)
        st.success("Modelo guardado exitosamente.")

        # Comprimir la carpeta del modelo
        namefile = datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '_modelo'
        shutil.make_archive(namefile, 'zip', modelo_dir)

        # Descargar el archivo comprimido
        with open(f"{namefile}.zip", "rb") as fp:
            btn = st.download_button(
                label="Descargar Modelo",
                data=fp,
                file_name=f"{namefile}.zip",
                mime="application/zip"
            )
    except Exception as e:
        st.error(f"Error al guardar el modelo: {e}")

else:
    st.warning("Aún no hay datos para guardar el modelo.")"""

'%%writefile pages/Entrenador.py\nimport streamlit as st\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport os\nimport subprocess\nimport yt_dlp as youtube_dl\nimport soundfile as sf\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.manifold import TSNE\nfrom sklearn.decomposition import PCA\nimport joblib\nimport gdown\nfrom datetime import datetime\nimport shutil\n\n# Configuración de la página\nst.set_page_config(\n    page_title="Análisis de Género Musical",\n    layout="wide",\n    initial_sidebar_state="expanded",\n)\n\nst.title("Análisis de Género Musical con Streamlit")\n\n# 1. Descarga de Datos desde Google Drive\nst.header("1. Descarga de Datos desde Google Drive")\n\nFILEID = "1DudgUNAZPWv_42o9iOEAKurQQlnuCPNL"\nDOWNLOAD_URL = f"https://drive.google.com/uc?id={FILEID}&export=download"\n\noutput = "canciones.xlsx"\n\nif not os.path.exists(output):\n    with st.spinner("Descargando archivo de Google Drive..."):\n        try:\n        

## Conexión a Ngrok

In [ ]:
#cambiar su token
usuario="Juan"
if usuario=="Yuliana":
  token = '2lLjHoDPtNqGFhA3XCMXaV7TSzC_2WMetgXt54NgDAv4qBuZE' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok
elif usuario=="Juan":
  token = '2lLkHdYMBU6KwzPcVAaxaNJyYYj_5y1Y8DSwG18FHt3fkdFzX'
elif usuario=='Cristian':
  token = '2lLjBy3NA0GhfCDLgRI47jQwMpL_4FDAUMmnURPshTKsBFXY8'

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run Parcial_3.py --server.port 8730 & #cambiar el puerto 8499 pueden poner cualquier número de 4 digitos

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='8730', proto='http', bind_tls=True) #acá tambien lo tienen que cambiar

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://7c24-35-199-56-91.ngrok-free.app


In [ ]:
import os
import subprocess

# Función para listar los túneles activos de ngrok
def listar_tuneles_ngrok():
    tuneles = subprocess.check_output(["ngrok", "tunnels"], text=True)
    print("Túneles activos:\n", tuneles)

# Función para cerrar un túnel específico
def cerrar_tunel_ngrok(tunnel_name):
    comando_cerrar = f"ngrok http -stop={tunnel_name}"
    os.system(comando_cerrar)
    print(f"Túnel {tunnel_name} cerrado.")

# Función para detener todos los túneles
def detener_ngrok():
    os.system("pkill ngrok")
    print("Todos los túneles de ngrok han sido detenidos.")

# Mostrar túneles activos
listar_tuneles_ngrok()

# Cerrar un túnel específico (reemplaza el ID del túnel por el tuyo)
cerrar_tunel_ngrok("tn_2mnPKN0oQEkgkKjXq5H7Tk5J6Xr")

# Detener todos los túneles (opción alternativa)
detener_ngrok()

Túneles activos:
 
Túnel tn_2mnPKN0oQEkgkKjXq5H7Tk5J6Xr cerrado.
Todos los túneles de ngrok han sido detenidos.
